In [ ]:
%load_ext jupyter_black

In [114]:
from __future__ import annotations

import time
import itertools
import random
import numpy as np

# from mesoscaler.generic import DataWorker, DataConsumer
import enum


True

In [ ]:
Index = tuple[int, int, int, int]


DUMMY_DATA: dict[Index, np.ndarray] = {
    # some dummy data either on disk or in
    k: np.random.rand(100, 100)
    for k in itertools.product([1, 2, 3], [4, 5, 6], [4, 5, 6], [4, 5, 6])
}


def get_data_from_disk(key: Index) -> np.ndarray:
    time.sleep(0.1)  # io latency
    return DUMMY_DATA[key]

In [ ]:
indices = random.choices(list(DUMMY_DATA.keys()), k=20)

print(len(DUMMY_DATA), len(indices))

The worker class is `Mapping` that is instantiated with a Iterable sequence of indices.

In [ ]:
class MyWorker(DataWorker[Index, np.ndarray]):
    def __getitem__(self, idx: Index) -> np.ndarray:
        return get_data_from_disk(idx)


worker = MyWorker(indices=indices)
print(worker[indices[0]])
worker

In [ ]:
from typing import Any, Generic, TypeVar, get_args
import types
import typing

K = TypeVar("K")
T = TypeVar("T")
import enum

Undefined = enum.Enum("", "Undefined")  # .Undefined
TypingGenericAlias = getattr(typing, "_GenericAlias")
print(type(TypingGenericAlias))


class FirstOrderGeneric(Generic[K, T]):
    __generic__: tuple[Any, ...]

    def __init_subclass__(cls) -> None:
        for arg in getattr(cls, "__orig_bases__", []):
            if isinstance(arg, TypingGenericAlias):
                cls.__generic__ = getattr(arg, "__args__", (Undefined,))
                break


class SomeBaseClass(FirstOrderGeneric[str, int], Generic[T]):
    ...


SomeBaseClass.__generic__  # prints "<class 'int'>"

In [ ]:
worker.to_dict()

In [ ]:
train, test = worker.split(0.8)
test

In [ ]:
from typing import get_type_hints

# get_type_hints(worker.__class__)

# DataConsumer

Assuming there is some IO bottle neck involved with loading data from disk the
`DataConsumer` can be used as a DataLoader that will queue up the data to be
loaded in the background while the model is training.

In [ ]:
start = time.time()
for idx in worker:
    data = worker[idx]
    time.sleep(0.1)
print("worker:", time.time() - start)

start = time.time()
for x in DataConsumer(worker):
    time.sleep(0.1)
print("consumer:", time.time() - start)